In [178]:
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report
import pandas as pd
import utils

In [179]:
dataset = pd.read_excel('data.xlsx')

In [180]:
dataset.shape

(2000, 6)

In [181]:
dataset.head()

,id_str,text,preprocessed_text,liwc_analysis,is_subjective,classificacao
0,1363503470377136130,@mvpcastro Nunca em momento houve limites no B...,nunca em momento houve limites no brasil para ...,"{'function (Function Words)': 12, 'adverb (Adv...",1,0.00
1,1364981948872089602,@tmaia29 @Flamengo porraaaaaa,porraaaaaa,{},0,nan
2,1363576501065707527,@avictaick Motivo o Flamengo?? Kkkkk,motivo o flamengo kkkkk,"{'cogproc (Cognitive Processes)': 1, 'insight ...",1,0.00
3,1364964990688575489,@Flamengo @AlexandreVidal1 👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼,NaN,{},0,nan
4,1363561902488637446,@negrixXxxXXx e quando n tem flamengo na parada,e quando n tem flamengo na parada,"{'function (Function Words)': 4, 'conj (Conjun...",0,nan


In [182]:
dataset.classificacao.value_counts()

0.00     819
1.00     272
-1.00     80
Name: classificacao, dtype: int64

In [183]:
dataset.is_subjective.value_counts()

1    1171
0     829
Name: is_subjective, dtype: int64

In [184]:
dataset = dataset.where(dataset.is_subjective == 1)

In [185]:
dataset.dropna(inplace=True)

In [186]:
dataset.drop_duplicates(subset=['preprocessed_text'], inplace=True)

In [187]:
dataset.rename(columns={"classificacao": "target"}, inplace=True)
dataset.target.replace(-1,1, inplace=True)
dataset = dataset[['preprocessed_text', 'target']]

In [188]:
dataset.preprocessed_text = dataset.preprocessed_text.apply(utils.remove_non_ascii_normalized)

In [189]:
#dataset.preprocessed_text = dataset.preprocessed_text.apply(utils.remove_stopwords)

In [190]:
X_train, X_test, y_train, y_test = train_test_split(dataset.preprocessed_text,
                               dataset.target, stratify=dataset.target, random_state=0)

In [191]:
X_train.shape, X_test.shape

((833,), (278,))

In [192]:
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1,1))

In [193]:
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [194]:
clf = LazyClassifier(verbose=3,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train_tfidf.A, X_test_tfidf.A, y_train, y_test)

  7%|▋         | 2/29 [00:01<00:21,  1.27it/s]{'Model': 'BaggingClassifier', 'Accuracy': 0.6870503597122302, 'Balanced Accuracy': 0.6001384124691581, 'ROC AUC': 0.6001384124691581, 'F1 Score': 0.6721026710640318, 'Time taken': 0.8697788715362549}
{'Model': 'BernoulliNB', 'Accuracy': 0.7338129496402878, 'Balanced Accuracy': 0.6372991514713847, 'ROC AUC': 0.6372991514713848, 'F1 Score': 0.7123680725749072, 'Time taken': 0.07770252227783203}
 21%|██        | 6/29 [00:06<00:22,  1.03it/s]{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.6870503597122302, 'Balanced Accuracy': 0.5, 'ROC AUC': 0.5, 'F1 Score': 0.5596017855225415, 'Time taken': 4.5941736698150635}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.60431654676259, 'Balanced Accuracy': 0.5305410122164049, 'ROC AUC': 0.5305410122164048, 'F1 Score': 0.60031017895551, 'Time taken': 0.14360260963439941}
 28%|██▊       | 8/29 [00:06<00:12,  1.64it/s]{'Model': 'DummyClassifier', 'Accuracy': 0.5863309352517986, 'Balanced Accuracy': 0.52

In [195]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LogisticRegression,0.74,0.69,0.69,0.74,0.14
PassiveAggressiveClassifier,0.72,0.67,0.67,0.72,0.11
GaussianNB,0.63,0.66,0.66,0.65,0.09
NearestCentroid,0.70,0.66,0.66,0.70,0.07
NuSVC,0.76,0.65,0.65,0.73,1.09
Perceptron,0.70,0.64,0.64,0.70,0.09
KNeighborsClassifier,0.69,0.64,0.64,0.69,0.51
SGDClassifier,0.75,0.64,0.64,0.72,0.10
BernoulliNB,0.73,0.64,0.64,0.71,0.08
